In [2]:
import pandas as pd


In [6]:
!pwd

/grid/kinney/home/desmara/MPSA_pred/SOTA_models/head_interpretation/SAE_Hackathon/data/whole_genome


In [26]:
gene_split = pd.read_csv('./canonical_gene_split.csv', index_col=[0])
display(gene_split.head())
gene_data = pd.read_csv('./gene_data.csv', index_col=[0])
display(gene_data.head())



,Unnamed: 0,gene,Test_set,CV0,CV1,CV2,CV3,CV4
0,0,ENSG00000091073,False,False,False,False,True,False
1,1,ENSG00000120616,False,True,False,False,False,False
2,2,ENSG00000158796,False,False,True,False,False,False
3,3,ENSG00000233006,True,False,False,False,False,False
4,4,ENSG00000175040,False,False,True,False,False,False


,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name
Gene stable ID,,,,,
ENSG00000284532,17,28360654,28360734,1,MIR4723
ENSG00000238933,9,61642383,61642494,1,RF00019
ENSG00000275693,10,88992370,88992539,1,RF02116
ENSG00000275451,15,62343029,62343138,1,MIR6085
ENSG00000222870,7,94495299,94495404,1,RNU6-1328P


In [28]:
to_chunk = pd.merge(left=gene_data, right=gene_split, left_index=True, right_on='gene', how='inner')
to_chunk.rename(columns={'gene': 'Gene stable ID'}, inplace=True)
to_chunk.set_index('Gene stable ID', inplace=True)
to_chunk

,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name,Unnamed: 0,Test_set,CV0,CV1,CV2,CV3,CV4
Gene stable ID,,,,,,,,,,,,
ENSG00000142552,19,49527618,49546962,1,RCN3,4929,False,False,False,True,False,False
ENSG00000112379,6,138161921,138344663,1,ARFGEF3,12327,False,False,False,True,False,False
ENSG00000103642,15,63121800,63142061,1,LACTB,7302,False,True,False,False,False,False
ENSG00000170855,12,120443961,120446412,-1,TRIAP1,10164,False,False,False,False,True,False
ENSG00000105732,19,42068477,42081549,1,ZNF574,1937,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000172250,22,42500579,42512560,1,SERHL,6108,False,False,True,False,False,False
ENSG00000203883,20,64047582,64049641,-1,SOX18,11209,False,False,True,False,False,False
ENSG00000163527,3,31532638,31637622,1,STT3B,448,False,False,False,False,False,True


In [36]:
def chunker(start, stop, window):
    j=None
    for i in range(start, stop, window):
        if j is not None:
            yield((j, i-1))
            j=i
        else:
            j=i

    if stop-j < window:
        yield(j,stop)
    else:
        mid = stop-(stop-j)//2
        yield(j,mid-1)
        yield(mid,stop)

def chunkify(to_chunk, chunk_size=5000):
    to_chunk['chunks'] = to_chunk.apply(lambda x: list(chunker(x['Gene start (bp)'],x['Gene end (bp)'], chunk_size)), axis=1)
    to_chunk = to_chunk.explode('chunks')
    to_chunk[['ChunkStart', 'ChunkEnd']] = pd.DataFrame(to_chunk['chunks'].tolist(), index=to_chunk.index)
    return(to_chunk)

gene_chunks = chunkify(to_chunk)
gene_chunks.head()

,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name,Unnamed: 0,Test_set,CV0,CV1,CV2,CV3,CV4,chunks,ChunkStart,ChunkEnd
Gene stable ID,,,,,,,,,,,,,,,
ENSG00000142552,19,49527618,49546962,1,RCN3,4929,False,False,False,True,False,False,"(49527618, 49532617)",49527618,49532617
ENSG00000142552,19,49527618,49546962,1,RCN3,4929,False,False,False,True,False,False,"(49532618, 49537617)",49532618,49537617
ENSG00000142552,19,49527618,49546962,1,RCN3,4929,False,False,False,True,False,False,"(49537618, 49542617)",49537618,49542617
ENSG00000142552,19,49527618,49546962,1,RCN3,4929,False,False,False,True,False,False,"(49542618, 49546962)",49542618,49546962
ENSG00000112379,6,138161921,138344663,1,ARFGEF3,12327,False,False,False,True,False,False,"(138161921, 138166920)",138161921,138166920


In [37]:
(gene_chunks['ChunkStart']<gene_chunks['ChunkEnd']).all()

np.True_

In [38]:
gene_chunks['ChunkEnd']-gene_chunks['ChunkStart']

Gene stable ID
ENSG00000142552    4999
ENSG00000142552    4999
ENSG00000142552    4999
ENSG00000142552    4344
ENSG00000112379    4999
                   ... 
ENSG00000187735    4999
ENSG00000187735    4999
ENSG00000187735    4999
ENSG00000187735    4999
ENSG00000187735     977
Length: 216107, dtype: int64

In [39]:
gene_chunks['AdjustedEnd'] = gene_chunks['ChunkEnd']+5000 - (gene_chunks['ChunkEnd']-gene_chunks['ChunkStart'])
gene_chunks

,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name,Unnamed: 0,Test_set,CV0,CV1,CV2,CV3,CV4,chunks,ChunkStart,ChunkEnd,AdjustedEnd
Gene stable ID,,,,,,,,,,,,,,,,
ENSG00000142552,19,49527618,49546962,1,RCN3,4929,False,False,False,True,False,False,"(49527618, 49532617)",49527618,49532617,49532618
ENSG00000142552,19,49527618,49546962,1,RCN3,4929,False,False,False,True,False,False,"(49532618, 49537617)",49532618,49537617,49537618
ENSG00000142552,19,49527618,49546962,1,RCN3,4929,False,False,False,True,False,False,"(49537618, 49542617)",49537618,49542617,49542618
ENSG00000142552,19,49527618,49546962,1,RCN3,4929,False,False,False,True,False,False,"(49542618, 49546962)",49542618,49546962,49547618
ENSG00000112379,6,138161921,138344663,1,ARFGEF3,12327,False,False,False,True,False,False,"(138161921, 138166920)",138161921,138166920,138166921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000187735,8,53966552,54022529,-1,TCEA1,3436,False,False,False,False,False,True,"(54001552, 54006551)",54001552,54006551,54006552
ENSG00000187735,8,53966552,54022529,-1,TCEA1,3436,False,False,False,False,False,True,"(54006552, 54011551)",54006552,54011551,54011552
ENSG00000187735,8,53966552,54022529,-1,TCEA1,3436,False,False,False,False,False,True,"(54011552, 54016551)",54011552,54016551,54016552


In [40]:
((gene_chunks['AdjustedEnd']-gene_chunks['ChunkStart'])==5000).all()

np.True_

In [52]:
gene_chunks['FlankStart'] = gene_chunks['ChunkStart']-5000
gene_chunks['FlankEnd'] = gene_chunks['AdjustedEnd']+5000
print(((gene_chunks['FlankEnd']-gene_chunks['FlankStart'])==15000).all())
gene_chunks

True


,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name,Unnamed: 0,Test_set,CV0,CV1,CV2,CV3,CV4,chunks,ChunkStart,ChunkEnd,AdjustedEnd,FlankStart,FlankEnd
Gene stable ID,,,,,,,,,,,,,,,,,,
ENSG00000142552,19,49527618,49546962,1,RCN3,4929,False,False,False,True,False,False,"(49527618, 49532617)",49527618,49532617,49532618,49522618,49537618
ENSG00000142552,19,49527618,49546962,1,RCN3,4929,False,False,False,True,False,False,"(49532618, 49537617)",49532618,49537617,49537618,49527618,49542618
ENSG00000142552,19,49527618,49546962,1,RCN3,4929,False,False,False,True,False,False,"(49537618, 49542617)",49537618,49542617,49542618,49532618,49547618
ENSG00000142552,19,49527618,49546962,1,RCN3,4929,False,False,False,True,False,False,"(49542618, 49546962)",49542618,49546962,49547618,49537618,49552618
ENSG00000112379,6,138161921,138344663,1,ARFGEF3,12327,False,False,False,True,False,False,"(138161921, 138166920)",138161921,138166920,138166921,138156921,138171921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000187735,8,53966552,54022529,-1,TCEA1,3436,False,False,False,False,False,True,"(54001552, 54006551)",54001552,54006551,54006552,53996552,54011552
ENSG00000187735,8,53966552,54022529,-1,TCEA1,3436,False,False,False,False,False,True,"(54006552, 54011551)",54006552,54011551,54011552,54001552,54016552
ENSG00000187735,8,53966552,54022529,-1,TCEA1,3436,False,False,False,False,False,True,"(54011552, 54016551)",54011552,54016551,54016552,54006552,54021552


In [53]:
gene_chunks.to_csv('./gene_chunks.csv', index=False)

In [60]:
gene_chunks.index.unique()

Index(['ENSG00000142552', 'ENSG00000112379', 'ENSG00000103642',
       'ENSG00000170855', 'ENSG00000105732', 'ENSG00000137955',
       'ENSG00000143224', 'ENSG00000165259', 'ENSG00000258504',
       'ENSG00000172602',
       ...
       'ENSG00000034053', 'ENSG00000101888', 'ENSG00000132881',
       'ENSG00000062282', 'ENSG00000144583', 'ENSG00000172250',
       'ENSG00000203883', 'ENSG00000163527', 'ENSG00000120451',
       'ENSG00000187735'],
      dtype='object', name='Gene stable ID', length=14375)

In [54]:
test_chunks = gene_chunks[gene_chunks.Test_set]
test_chunks.to_csv('./test_chunks.csv', index=False)

In [55]:
val_chunks = gene_chunks[gene_chunks.CV0]
val_chunks.to_csv('./val_chunks.csv', index=False)
train_chunks = gene_chunks[~gene_chunks.Test_set & ~gene_chunks.CV0]
train_chunks.to_csv('./train_chunks.csv', index=False)
